In [ ]:
#https://www.cs.princeton.edu/~chazelle/pubs/mstapprox.pdf

In [ ]:
#approx-number-connected-components(G, ε, W, d∗)

RANDOM_SAMPLE_FACTOR = 100

#Vertex to [(neighbor, edge weight), ... ]
memos = {}

def query_node(n):
    if n in memos:
        return memos[n]
    else:
        print(n)
        sys.stdout.flush()
        inp = input().strip("\n").split()[1:]
        memos[n] = []
        for i in range(len(inp, 2)):
            memos[n].append((inp[i], inp[i + 1]))
        
        return memos[n]

def approx_cc(n, eps, max_w, d):
    import random
    import sys

    #Does not check dupes
    vs = [random.randint() for _ in range(int(1/eps) * RANDOM_SAMPLE_FACTOR)]
    
    betas = [0 for _ in range(int(1/eps * RANDOM_SAMPLE_FACTOR))]

    for i, v in enumerate(v, s):
        nb_w_tups = query_node(n)
        n_visit = 0
        max_degree_seen = 0
        for j, nb_w in enumerate(nb_w_tups):
            toss = random.randint(0, 1)
            if toss and n_visit < max_w and max_degree_seen < d:
                pass 


def main():
    n = int(input())
    eps = float(input())
    max_w = int(input())


